In [ ]:
import os
import gc
import sys
import time
import numpy as np
import pandas as pd
import cv2
import PIL.Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from apex import amp
import torchvision
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from efficientnet_pytorch import model as enet
import albumentations
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
from tqdm.notebook import tqdm
DEVICE = torch.device('cuda')
for i in range(torch.cuda.device_count()):
    print(i, torch.cuda.get_device_name(i))
    print('  allocated:', round(torch.cuda.memory_allocated(i) / 1024 ** 3, 1), 'GB')
    print('  cached:   ', round(torch.cuda.memory_cached(i) / 1024 ** 3, 1), 'GB')

In [ ]:
MAIN_PATH = '.'
DATA_PATH = '{}/data'.format(MAIN_PATH)
LABELS_PATH = '{}/train.csv'.format(DATA_PATH)
DEBUG = False
APEX = True
M_PARAMS = {
    'MODEL_VER': 'v28',
    'TILES_DIR': '{}/train_images_tiles_q1_256_36/'.format(DATA_PATH),
    'ENET_TYPE': 'efficientnet-b1',
    'N_FOLDS': 5,
    'TILE_SIZE': 256,
    'N_TILES': 25,
    'BATCH_SIZE': 6,
    'N_WORKERS': 12,
    'OUT_DIM': 5,
    'LR': 3e-4,
    'N_EPOCHS': 10 if DEBUG else 30,
    'WARMUP': True,
    'SEED': 2020,
    'DROPOUT': .4,
    'RAND': False,
    'COMMENTS': 'only affine effnet_ no bn double drop'
}
print(M_PARAMS)
MODELS_PATH = '{}/effnet_models_{}'.format(MAIN_PATH, M_PARAMS['MODEL_VER'])
if not os.path.exists(MODELS_PATH):
    os.mkdir(MODELS_PATH)
    print('created:', MODELS_PATH)
PRETRAINED_MODEL = {
    'efficientnet-b0': './bbs/efficientnet-b0-355c32eb.pth',
    'efficientnet-b1': './bbs/efficientnet-b1-f1951068.pth',
    'efficientnet-b2': './bbs/efficientnet-b2-8bb594d6.pth',
    'efficientnet-b3': './bbs/efficientnet-b3-5fb5a3c3.pth'
}

In [ ]:
if DEBUG:
    df_train = pd.read_csv(os.path.join(DATA_PATH, 'train_dsph.csv')).sample(100).reset_index(drop=True)
else:
    df_train = pd.read_csv(os.path.join(DATA_PATH, 'train_dsph.csv'))
df_train.head()

In [ ]:
skf = StratifiedKFold(M_PARAMS['N_FOLDS'], shuffle=True, random_state=M_PARAMS['SEED'])
df_train['fold'] = -1
for i, (train_idx, valid_idx) in enumerate(skf.split(df_train, df_train['isup_grade'])):
    df_train.loc[valid_idx, 'fold'] = i
df_train.head()

In [ ]:
class EffNet(nn.Module):
    def __init__(self, backbone, out_dim):
        super(EffNet, self).__init__()
        self.enet = enet.EfficientNet.from_name(backbone)
        self.enet.load_state_dict(torch.load(PRETRAINED_MODEL[backbone]))
        #self.enet = enet.EfficientNet.from_pretrained(backbone)
        nc = self.enet._fc.in_features
        self.myfc = nn.Linear(nc, out_dim)
        self.enet._fc = nn.Identity()
        self.enet = nn.DataParallel(self.enet)
    def forward(self, x):
        x = self.enet(x)
        x = self.myfc(x)
        return x
class EffNet_(nn.Module):
    def __init__(self, backbone, out_dim):
        super(EffNet_, self).__init__()
        self.enet = enet.EfficientNet.from_name(backbone)
        self.enet.load_state_dict(torch.load(PRETRAINED_MODEL[backbone]))
        nc = self.enet._fc.in_features
        self.enet._fc = nn.Identity()
        self.enet = nn.DataParallel(self.enet)
        self.myfc = nn.DataParallel(
            nn.Sequential(
                nn.Dropout(M_PARAMS['DROPOUT']),
                nn.Linear(nc, int(nc / 4)),
                nn.ELU(),
                #nn.BatchNorm1d(int(nc / 4)), 
                nn.Dropout(M_PARAMS['DROPOUT']),
                nn.Linear(int(nc / 4), out_dim)
            )
        )
    def extract(self, x):
        return self.enet(x)
    def forward(self, x):
        x = self.extract(x)
        x = self.myfc(x)
        return x
class ResNext(nn.Module):
    def __init__(self, out_dim):
        super(ResNext, self).__init__()
        self.rsnxt = torchvision.models.resnext50_32x4d(pretrained=True)
        nc = self.rsnxt.fc.in_features
        self.rsnxt.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(nc, int(nc / 4)),
            nn.ReLU(),
            nn.BatchNorm1d(int(nc / 4)), 
            nn.Dropout(.4),
            nn.Linear(int(nc / 4), out_dim)
        )
        self.rsnxt = nn.DataParallel(self.rsnxt)
    def forward(self, x):
        x = self.rsnxt(x)
        return x

In [ ]:
def get_tiles(img_id, n_tiles):
        result = []
        seq_imgs = []
        for i in range(n_tiles):
            img_path = '{}/{}_{}.png'.format(M_PARAMS['TILES_DIR'], img_id, i)
            img = cv2.imread(img_path)
            if not np.any(img):
                print('no img file read:', img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            seq_imgs.append(img)
        for i in range(len(seq_imgs)):
            result.append({'img':seq_imgs[i], 'idx':i})
        return result
class PANDADataset(Dataset):
    def __init__(self, df, tile_size, n_tiles, tile_mode=0, rand=False,
                 transform=None, transform_tile=None):
        self.df = df.reset_index(drop=True)
        self.tile_size = tile_size
        self.n_tiles = n_tiles
        self.tile_mode = tile_mode
        self.rand = rand
        self.transform = transform
        self.transform_tile = transform_tile
    def __len__(self):
        return self.df.shape[0]
    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_id = row.image_id
        tiles = get_tiles(img_id, self.n_tiles)
        if self.rand:
            idxes = np.random.choice(list(range(self.n_tiles)), self.n_tiles, replace=False)
        else:
            idxes = list(range(self.n_tiles))
        n_row_tiles = int(np.sqrt(self.n_tiles))
        images = np.zeros((self.tile_size * n_row_tiles, self.tile_size * n_row_tiles, 3))
        for h in range(n_row_tiles):
            for w in range(n_row_tiles):
                i = h * n_row_tiles + w
                if len(tiles) > idxes[i]:
                    this_img = tiles[idxes[i]]['img']
                else:
                    this_img = np.ones((self.tile_size, self.tile_size, 3)).astype(np.uint8) * 255
                this_img = (255 - this_img) / 255
                if self.transform_tile is not None:
                    this_img = self.transform_tile(image=this_img)['image']
                h1 = h * self.tile_size
                w1 = w * self.tile_size
                images[h1 : h1 + self.tile_size, w1:w1 + self.tile_size] = this_img
        images = images.astype(np.float32)
        if self.transform is not None:
            images = self.transform(image=images)['image']
        images = images.transpose(2, 0, 1)
        label = np.zeros(5).astype(np.float32)
        label[:row.isup_grade] = 1.
        return torch.tensor(images), torch.tensor(label)

In [ ]:
transforms_train = albumentations.Compose(
    [
        #albumentations.OneOf(
        #    [
        #        albumentations.RandomBrightness(limit=.1), 
        #        albumentations.RandomContrast(limit=.1), 
        #        albumentations.RandomGamma()
        #    ], 
        #    p=.33
        #),
        albumentations.Transpose(p=.5),
        albumentations.VerticalFlip(p=.5),
        albumentations.HorizontalFlip(p=.5),
        albumentations.Rotate(limit=15, p=.33)
    ]
)
transforms_train_tile = albumentations.Compose(
    [
        albumentations.Transpose(p=.5),
        albumentations.VerticalFlip(p=.5),
        albumentations.HorizontalFlip(p=.5)
    ]
)

In [ ]:
dataset_show = PANDADataset(
    df_train,
    M_PARAMS['TILE_SIZE'], 
    M_PARAMS['N_TILES'], 
    tile_mode=0, 
    transform=transforms_train,
    transform_tile=transforms_train_tile,
    rand=M_PARAMS['RAND']
)
from pylab import rcParams
rcParams['figure.figsize'] = (16, 4)
f, axarr = plt.subplots(1, M_PARAMS['BATCH_SIZE'])
for i in range(M_PARAMS['BATCH_SIZE']):
    img, label = dataset_show.__getitem__(i)
    axarr[i].imshow(1. - img.transpose(0, 1).transpose(1,2).squeeze())
    axarr[i].set_title(str(sum(label)))

In [ ]:
criterion = nn.BCEWithLogitsLoss()
def train_epoch(loader, optimizer):
    model.train()
    train_loss = []
    bar = tqdm(loader)
    for (data, target) in bar:
        data, target = data.to(DEVICE), target.to(DEVICE)
        loss_func = criterion
        optimizer.zero_grad()
        logits = model(data)
        loss = loss_func(logits, target)
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        #loss.backward()
        optimizer.step()
        loss_np = loss.detach().cpu().numpy()
        train_loss.append(loss_np)
        smooth_loss = sum(train_loss[-100:]) / min(len(train_loss), 100)
        bar.set_description('loss: %.5f, smth: %.5f' % (loss_np, smooth_loss))
    return train_loss
def val_epoch(loader, get_output=False):
    model.eval()
    val_loss = []
    LOGITS = []
    PREDS = []
    TARGETS = []
    with torch.no_grad():
        for (data, target) in tqdm(loader):
            data, target = data.to(DEVICE), target.to(DEVICE)
            logits = model(data)
            loss = criterion(logits, target)
            pred = logits.sigmoid().sum(1).detach().round()
            LOGITS.append(logits)
            PREDS.append(pred)
            TARGETS.append(target.sum(1))
            val_loss.append(loss.detach().cpu().numpy())
        val_loss = np.mean(val_loss)
    LOGITS = torch.cat(LOGITS).cpu().numpy()
    PREDS = torch.cat(PREDS).cpu().numpy()
    TARGETS = torch.cat(TARGETS).cpu().numpy()
    acc = (PREDS == TARGETS).mean() * 100.
    qwk = cohen_kappa_score(PREDS, TARGETS, weights='quadratic')
    qwk_k = cohen_kappa_score(
        PREDS[df_valid['data_provider'] == 'karolinska'], 
        df_valid[df_valid['data_provider'] == 'karolinska'].isup_grade.values, 
        weights='quadratic'
    )
    qwk_r = cohen_kappa_score(
        PREDS[df_valid['data_provider'] == 'radboud'], 
        df_valid[df_valid['data_provider'] == 'radboud'].isup_grade.values, 
        weights='quadratic'
    )
    print('QWK', qwk, 'QWK_k', qwk_k, 'QWK_r', qwk_r)
    if get_output:
        return LOGITS
    else:
        return val_loss, acc, qwk

In [ ]:
pred, target = [], []
if DEBUG:
    n_folds_train = 2
else:
    n_folds_train = M_PARAMS['N_FOLDS']
start_folds_train = 0
for fold_num in range(start_folds_train, n_folds_train):
    print('-' * 20, 'fold:', fold_num, '-' * 20)
    train_idx = np.where((df_train['fold'] != fold_num))[0]
    valid_idx = np.where((df_train['fold'] == fold_num))[0]
    df_this  = df_train.loc[train_idx]
    df_valid = df_train.loc[valid_idx]
    dataset_train = PANDADataset(
        df_this, 
        M_PARAMS['TILE_SIZE'], 
        M_PARAMS['N_TILES'], 
        transform=transforms_train,
        transform_tile=transforms_train_tile,
        rand=M_PARAMS['RAND']
    )
    dataset_valid = PANDADataset(
        df_valid, 
        M_PARAMS['TILE_SIZE'], 
        M_PARAMS['N_TILES']
    )
    train_loader = torch.utils.data.DataLoader(
        dataset_train, 
        batch_size=M_PARAMS['BATCH_SIZE'], 
        sampler=RandomSampler(dataset_train), 
        num_workers=M_PARAMS['N_WORKERS']
    )
    valid_loader = torch.utils.data.DataLoader(
        dataset_valid, 
        batch_size=M_PARAMS['BATCH_SIZE'], 
        sampler=SequentialSampler(dataset_valid), 
        num_workers=M_PARAMS['N_WORKERS']
    )
    model = EffNet_(M_PARAMS['ENET_TYPE'], out_dim=M_PARAMS['OUT_DIM']) 
    #model = ResNext(out_dim=M_PARAMS['OUT_DIM'])
    model = model.to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=M_PARAMS['LR'])
    model, optimizer = amp.initialize(model, optimizer, opt_level='O1')
    if M_PARAMS['WARMUP']:
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer, 
            max_lr=M_PARAMS['LR'], 
            total_steps=M_PARAMS['N_EPOCHS'],
            div_factor=(M_PARAMS['LR'] / 1e-5), 
            final_div_factor=1000,
            pct_start=(2 / M_PARAMS['N_EPOCHS']),
        )
    else:
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, M_PARAMS['N_EPOCHS'])
    print('train len:', len(dataset_train),'| val len:', len(dataset_valid))
    qwk_max = 0
    best_file = '{}/{}_best_fold{}.pth'.format(MODELS_PATH, M_PARAMS['ENET_TYPE'], fold_num)
    for epoch in tqdm(range(M_PARAMS['N_EPOCHS']), desc='epochs'):
        print(time.ctime(), 'epoch:', epoch)
        train_loss = train_epoch(train_loader, optimizer)
        val_loss, acc, qwk = val_epoch(valid_loader)
        scheduler.step(epoch)
        content = '{} epoch {}, lr: {:.8f}, train loss: {:.4f}, val loss: {:.4f}, acc: {:.1f}, QWK: {:.4f}'.format(
                time.ctime(),
                epoch, 
                optimizer.param_groups[0]['lr'], 
                np.mean(train_loss),
                np.mean(val_loss),
                acc,
                qwk
            )
        print(content)
        with open('{}/log_{}_fold{}.txt'.format(MODELS_PATH, M_PARAMS['ENET_TYPE'], fold_num), 'a') as appender:
            appender.write(content + '\n')
        if qwk > qwk_max:
            torch.save(model.state_dict(), best_file)
            print('QWK improved {:.6f} --> {:.6f} model saved'.format(qwk_max, qwk))
            qwk_max = qwk
    with open('{}/log_{}_folds_all.txt'.format(MODELS_PATH, M_PARAMS['ENET_TYPE']), 'a') as appender:
        appender.write('{} | fold: {} | max QWK: {:.6f}\n'.format(M_PARAMS, fold_num, qwk_max))
    torch.save(
        model.state_dict(), 
        os.path.join('{}/{}_final_fold{}.pth'.format(MODELS_PATH, M_PARAMS['ENET_TYPE'], fold_num))
    )
    del model, dataset_train, dataset_valid, train_loader, valid_loader
    torch.cuda.empty_cache()
    gc.collect()